### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
# import data

In [ ]:
info = pd.read_excel("data_dictionary.xlsx")

In [ ]:
info

In [ ]:
survey= pd.read_csv("employee_survey_data.csv")
survey.columns= survey.columns.str.lower()
survey

In [ ]:
#check for nan values
survey.isnull().values.any()

In [ ]:
display(survey.isnull().sum())

In [ ]:
general= pd.read_csv('general_data.csv')
general
general.columns= general.columns.str.lower()
general

In [ ]:
general["employeecount"].unique()

In [ ]:
general["over18"].value_counts()

In [ ]:
manager= pd.read_csv('manager_survey_data.csv')
manager.columns= manager.columns.str.lower()

In [ ]:
general["over18"].unique()

In [ ]:
# merge data sets

In [ ]:
general= pd.merge(general, survey, on = "employeeid", how = "inner")
general

In [ ]:
general= pd.merge(general, manager, on = "employeeid", how = "inner")

In [ ]:
general

In [ ]:
general.dtypes

In [ ]:
general['jobsatisfaction'].unique()

In [ ]:
general['totalworkingyears'].unique()

In [ ]:
display(general.isnull().sum())

### Data Cleaning

#### Replacing null value with 0

In [ ]:
np.where(general['numcompaniesworked'].isnull())[0]

In [ ]:
general['numcompaniesworked'] = general['numcompaniesworked'].fillna(value = 0)

In [ ]:
general['numcompaniesworked'].isnull().sum()

In [ ]:
general['numcompaniesworked'].value_counts()

#### Replacing null value with mode

In [ ]:
np.where(general['environmentsatisfaction'].isnull())[0]

In [ ]:
for column in ['environmentsatisfaction']:
    general[column].fillna(general[column].mode()[0], inplace=True)

In [ ]:
general['environmentsatisfaction'].isnull().sum()

#### Replacing null value with mode

In [ ]:
np.where(general['jobsatisfaction'].isnull())[0]

In [ ]:
for column in ['jobsatisfaction']:
    general[column].fillna(general[column].mode()[0], inplace=True)

In [ ]:
general['jobsatisfaction'].isnull().sum()

#### Replacing null value with mode

In [ ]:
np.where(general['worklifebalance'].isnull())[0]

In [ ]:
for column in ['worklifebalance']:
    general[column].fillna(general[column].mode()[0], inplace=True)

In [ ]:
general['worklifebalance'].isnull().sum()

#### Replacing null value with mode totalworkingyears

In [ ]:
for column in ['totalworkingyears']:
    general[column].fillna(general[column].mode()[0], inplace=True)

In [ ]:
general.isnull().sum()

#### Handling Null Value

In [ ]:
for column in general.columns:
    if general[column].dtype == 'float64':
        general[column] = general[column].astype('int64')

In [ ]:
general.dtypes

#### Dropping not informative columns

In [ ]:
general['over18'].value_counts()

In [ ]:
general['employeecount'].value_counts()

In [ ]:
general['standardhours'].value_counts()

In [ ]:
general = general.drop(['over18','employeecount', 'standardhours' ], axis = 1).reset_index(drop = True)


In [ ]:
general

### Transforming dt Time

In [ ]:
in_time= pd.read_csv('in_time.csv')
in_time.head()

In [ ]:
out_time = pd.read_csv('out_time.csv')
out_time.head()

In [ ]:
## Concat time data sets

In [ ]:
total_time= in_time.append(out_time)
total_time

In [ ]:
# remove 0 values (0 values are days where the office was closed)


In [ ]:
total_time = total_time.dropna(axis=1, how='all')

In [ ]:
total_time

In [ ]:
# replace individual days that have NaN with 0 ( for future calculations)

In [ ]:
total_time.fillna(0, inplace= True)
total_time

In [ ]:
## Convert to datatime format

In [ ]:
total_time.iloc[:,1:]= total_time.iloc[:,1:].apply(pd.to_datetime, errors='coerce') 

In [ ]:
total_time

In [ ]:
total_time = total_time.diff(periods=4410)

In [ ]:
total_time = total_time.iloc[4410:]

In [ ]:
total_time

In [ ]:
total_time.reset_index(inplace=True)

In [ ]:
total_time

In [ ]:
total_time.drop(columns=['index','Unnamed: 0', ''],axis=1,inplace=True)

In [ ]:
total_time.head()

In [ ]:
total_time['mean_time'] = total_time.mean(axis=1)

In [ ]:
total_time

In [ ]:
# convert mean time into hours
total_time.mean_time = total_time.mean_time.apply(lambda x : int(str(x).split(' ')[2].split(':')[0]) + int(str(x).split(' ')[2].split(':')[1])/60 + float(str(x).split(' ')[2].split(':')[2])/3600)
total_time.index = range(1,len(total_time)+1)

In [ ]:
total_time.reset_index(inplace=True)

In [ ]:
total_time

In [ ]:
total_time.index = range(1,len(total_time)+1)

In [ ]:
# Rename index as emID to add the general data set
total_time = total_time.rename(columns = {'index' : 'employeeid'})

In [ ]:
# Overwrite totalt only with the columns we need for general
total_time = total_time[['employeeid','mean_time']]  


In [ ]:
general = pd.merge(general, total_time, on = 'employeeid', how = 'inner')

In [ ]:
general

In [ ]:
general.to_csv('general_hr_data.csv')